# Trabalho 1 - EEL891

Etapas que serão realizadas a seguir:
- Implementar o pré-processamento
- Visualizar, analisar e selecionar os atributos
- Escolher os métodos que serão usados e compará-los
- Verificar o desempenho com validação cruzada
- Ajustas os hiperparâmetros para chegar ao melhor resultado possível
- Treinar o modelo com o conjunto de teste

### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy  as np

from matplotlib                 import pyplot as plt

from sklearn.naive_bayes        import MultinomialNB, GaussianNB
from sklearn.model_selection    import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics            import accuracy_score
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.preprocessing      import LabelBinarizer, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.linear_model       import LogisticRegression
from scipy.stats                import pearsonr

import sklearn

### Pré-processamento

Agora, o próximo passo é importar o conjunto de treinamento:

In [2]:
dados = pd.read_csv(r"C:\Users\andre\Documents\UFRJ\SÉTIMO PERÍODO\Introdução ao Aprendizado de Máquina\EEL891-Trabalho-1\conjunto_de_treinamento.csv")
dados = dados.iloc[:,:]

Embaralhamento dos dados e separá-los em atributos e alvo (convertendo para um array):

In [3]:
dados = dados.sample(frac=1)
x = dados.iloc[:,:-1].to_numpy()
y = dados.iloc[:,-1].to_numpy()

Descartando algumas informações que não são relevantes. O campo "id_solicitante" é um número de 1 a 20000, ou seja, apenas atrapalharia o modelo. Os campos "grau_instrucao" e "possui_telefone_celular" não foram extraídos corretamente, então segui a orientação do professor para descartá-los. E o campo "qtde_contas_bancarias_especiais" é exatamente igual ao campo "qtde_contas_bancarias", logo foi descartado seguindo a orientação do professor.

In [4]:
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'possui_telefone_celular', 'qtde_contas_bancarias_especiais'],axis=1)

Identificação das variáveis categóricas olhando o dicionário de dados:

In [5]:
variaveisCategoricas = [x for x in dados.columns if dados[x].dtype=='object' or x == 'produto_solicitado' or x == 'tipo_endereco' or x =='estado_civil' or  x =='nacionalidade' or x=='tipo_residencia' or x=='possui_email' or x=='possui_cartao_visa' or x=='possui_cartao_mastercard' or x=='possui_cartao_diners' or x=='possui_cartao_amex' or x=='possui_cartao_cartoes' or x=='possui_carro' or x=='profissao' or x=='ocupacao' or x=='profissao_companheiro' or x=='local_onde_reside' or x=='local_onde_trabalha']
print(variaveisCategoricas)

['produto_solicitado', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial', 'codigo_area_telefone_residencial', 'tipo_residencia', 'possui_email', 'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_cartao_diners', 'possui_cartao_amex', 'possui_carro', 'vinculo_formal_com_empresa', 'estado_onde_trabalha', 'possui_telefone_trabalho', 'codigo_area_telefone_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro', 'local_onde_reside', 'local_onde_trabalha']


Verificando as variáveis categóricas (quantidade de categorias e quais são):

In [6]:
for v in variaveisCategoricas:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")


produto_solicitado:     3 categorias
[1 7 2] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     4 categorias
['F' 'M' 'N' ' '] 


   estado_civil:     8 categorias
[4 1 2 6 3 5 7 0] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['RS' 'SP' 'PB' 'BA' 'DF' 'CE' 'RJ' 'AP' 'MG' 'PE' 'MT' 'MA' 'ES' 'SC'
 'PR' ' ' 'SE' 'PA' 'RN' 'GO' 'AL' 'MS' 'AC' 'PI' 'AM' 'RO' 'TO' 'RR'] 


estado_onde_reside:    27 categorias
['RR' 'RS' 'SP' 'BA' 'GO' 'CE' 'PE' 'AP' 'MG' 'MT' 'RJ' 'PA' 'SC' 'PR'
 'ES' 'MS' 'RN' 'AL' 'AC' 'DF' 'PB' 'MA' 'RO' 'PI' 'AM' 'TO' 'SE'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['54' '58' '5' ' ' '67' '107' '97' '123' '32' '12' '30' '69' '84' '9' '20'
 '81' '50' '15' '112' '47' '25' '29' '75' '40' '71' '105' '18' '100' '76'
 '66' '103' '33' '16' '125' '77' '41' '110' '86' '22'

Correção de alguns valores (principalmente em branco). A ideia inicial é criar uma nova categoria para os valores em branco, e depois tentar excluí-los para ver qual é a resposta dos modelos:

In [7]:
for x in variaveisCategoricas:
    if x == 'sexo':
        dados[x].replace(' ', 'N', inplace=True)
    if x == 'estado_onde_nasceu':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_residencial':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'tipo_residencia':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'estado_onde_trabalha':
        dados[x].replace(' ', 'NN', inplace=True)
    if x == 'codigo_area_telefone_trabalho':
        dados[x].replace(' ', '000', inplace=True)
    if x == 'profissao':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'ocupacao':
        dados[x].replace(0, 6, inplace=True)
        dados[x].fillna(0, inplace=True)
    if x == 'profissao_companheiro':
        dados[x].replace(0, 18, inplace=True)
        dados[x].fillna(0, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[1 7 2] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['F' 'M' 'N'] 


   estado_civil:     8 categorias
[4 1 2 6 3 5 7 0] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['RS' 'SP' 'PB' 'BA' 'DF' 'CE' 'RJ' 'AP' 'MG' 'PE' 'MT' 'MA' 'ES' 'SC'
 'PR' 'NN' 'SE' 'PA' 'RN' 'GO' 'AL' 'MS' 'AC' 'PI' 'AM' 'RO' 'TO' 'RR'] 


estado_onde_reside:    27 categorias
['RR' 'RS' 'SP' 'BA' 'GO' 'CE' 'PE' 'AP' 'MG' 'MT' 'RJ' 'PA' 'SC' 'PR'
 'ES' 'MS' 'RN' 'AL' 'AC' 'DF' 'PB' 'MA' 'RO' 'PI' 'AM' 'TO' 'SE'] 


possui_telefone_residencial:     2 categorias
['Y' 'N'] 


codigo_area_telefone_residencial:    81 categorias
['54' '58' '5' '000' '67' '107' '97' '123' '32' '12' '30' '69' '84' '9'
 '20' '81' '50' '15' '112' '47' '25' '29' '75' '40' '71' '105' '18' '100'
 '76' '66' '103' '33' '16' '125' '77' '41' '110' '86' '22' 

In [8]:
for x in dados:
    if x == 'meses_na_residencia':
        dados[x].fillna(300, inplace=True)
    if x == 'grau_instrucao_companheiro':
        dados[x].fillna(6, inplace=True)
    print("\n%15s: " %x, "%4d categorias" % len(dados[x].unique() ) )
    print(dados[x].unique(), "\n")


produto_solicitado:     3 categorias
[1 7 2] 


 dia_vencimento:     6 categorias
[ 5 25 10  1 20 15] 


forma_envio_solicitacao:     3 categorias
['internet' 'presencial' 'correio'] 


  tipo_endereco:     2 categorias
[1 2] 


           sexo:     3 categorias
['F' 'M' 'N'] 


          idade:    84 categorias
[ 68  18  19  51  55  41  27  52  34  35  26  62  42  50  84  40  23  43
  44  32  57  33  29  31  58  37  59  54  46  69  67  48  74  39  61  38
  28  83  65  36  30  64  49  22  24  80  72  45  25  90  78  53  60  66
  21  56  20  71  63  73  70  47  77  85  76  81  75  88  82  86  95  93
  79  91  92 100   7  89  87 106  17  94  96  97] 


   estado_civil:     8 categorias
[4 1 2 6 3 5 7 0] 


qtde_dependentes:    15 categorias
[ 0  1  6  3  5  4  2  7  8  9 13 53 10 11 14] 


  nacionalidade:     3 categorias
[1 0 2] 


estado_onde_nasceu:    28 categorias
['RS' 'SP' 'PB' 'BA' 'DF' 'CE' 'RJ' 'AP' 'MG' 'PE' 'MT' 'MA' 'ES' 'SC'
 'PR' 'NN' 'SE' 'PA' 'RN' 'GO' 'AL' 'MS' 'AC' '

Binarizar e fazer o one hot encoding de alguns parâmetros:

In [9]:
dados = pd.get_dummies(dados, columns=['forma_envio_solicitacao',
                                       'sexo'
                                       #'estado_onde_nasceu',
                                       #'estado_onde_reside',
                                       #'estado_onde_trabalha'
                                      ])
#print(dados.T)

binarizador = LabelBinarizer()
for x in ['tipo_endereco',
          'possui_telefone_residencial',
          'vinculo_formal_com_empresa',
          'possui_telefone_trabalho']:
    dados[x] = binarizador.fit_transform(dados[x])
print(dados.T)

                                    2037   15298  19597  3270    4494   5417   \
produto_solicitado                      1      1      1      1       7      1   
dia_vencimento                          5      5     25     25      25      5   
tipo_endereco                           0      0      0      0       0      0   
idade                                  68     18     19     51      55     41   
estado_civil                            4      1      2      2       6      2   
qtde_dependentes                        0      0      0      1       6      1   
nacionalidade                           1      1      1      1       1      1   
estado_onde_nasceu                     RS     RS     SP     PB      BA     RS   
estado_onde_reside                     RR     RS     SP     SP      BA     RS   
possui_telefone_residencial             1      1      1      1       0      1   
codigo_area_telefone_residencial       54     58      5      5     000     54   
tipo_residencia             

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [10]:
colunas = dados.columns
for col in colunas:
    if type(dados[col][0]) != str:
        corr = pearsonr(dados[col], dados['inadimplente'])
        print(col, " = %.4f" % corr[0], " , %.10f" % corr[1])

produto_solicitado  = 0.0301  , 0.0000206384
dia_vencimento  = 0.0803  , 0.0000000000
tipo_endereco  = -0.0044  , 0.5332219466
idade  = -0.1207  , 0.0000000000
estado_civil  = -0.0313  , 0.0000097351
qtde_dependentes  = 0.0176  , 0.0126681749
nacionalidade  = 0.0005  , 0.9442714564
possui_telefone_residencial  = -0.0808  , 0.0000000000
tipo_residencia  = 0.0200  , 0.0046491853
meses_na_residencia  = -0.0047  , 0.5024577844
possui_email  = -0.0057  , 0.4168413604
renda_mensal_regular  = -0.0009  , 0.8958507584
renda_extra  = 0.0059  , 0.4079845863
possui_cartao_visa  = -0.0013  , 0.8559114105
possui_cartao_mastercard  = -0.0200  , 0.0047680534
possui_cartao_diners  = 0.0025  , 0.7234775228
possui_cartao_amex  = -0.0011  , 0.8757773534
possui_outros_cartoes  = -0.0032  , 0.6543764344
qtde_contas_bancarias  = 0.0141  , 0.0456463167
valor_patrimonio_pessoal  = 0.0012  , 0.8627919655
possui_carro  = 0.0128  , 0.0707060897
vinculo_formal_com_empresa  = 0.0062  , 0.3773057238
possui_telefone_

In [11]:
#atributos usados pelo classificador
atributosSelecionados = [
    'produto_solicitado',
    'dia_vencimento',
    'forma_envio_solicitacao_correio',
    'forma_envio_solicitacao_internet',
    'forma_envio_solicitacao_presencial',
    'tipo_endereco',
    'sexo_M',
    'sexo_F',
    'sexo_N',
    'idade',
    'estado_civil',
    'qtde_dependentes',
    'nacionalidade',
    #'estado_onde_nasceu',
    #'estado_onde_reside',
    'possui_telefone_residencial',
    #'codigo_area_telefone_residencial',
    'tipo_residencia',
    'meses_na_residencia',
    'possui_email',
    'renda_mensal_regular',
    'renda_extra',
    'possui_cartao_visa',
    'possui_cartao_mastercard',
    'possui_cartao_diners',
    'possui_cartao_amex',
    'possui_outros_cartoes',
    'qtde_contas_bancarias',
    'valor_patrimonio_pessoal',
    'possui_carro',
    'vinculo_formal_com_empresa',
    #'estado_onde_trabalha',
    'possui_telefone_trabalho',
    #'codigo_area_telefone_trabalho',
    'meses_no_trabalho',
    'profissao',
    'ocupacao',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    #'local_onde_reside',
    #'local_onde_trabalha',
    'inadimplente'
]

alvo = 'inadimplente'

#embaralhar o conjunto de dados
dadosEmbaralhados = dados[atributosSelecionados].sample(frac=1)

#criar arrays x e y separando os parâmetros do alvo
x = dadosEmbaralhados.iloc[:,:-1].to_numpy()
y = dadosEmbaralhados.iloc[:,-1].to_numpy()

# #separar em conjunto de treino e de teste
# amostrasTreinamento = 15000

# xTreino = x[:amostrasTreinamento,:]
# yTreino = y[:amostrasTreinamento].ravel()

# xTeste = x[amostrasTreinamento:,:]
# yTeste = y[amostrasTreinamento:].ravel()

# #ajustar a escala
# scaler = MinMaxScaler()
# scaler.fit(xTreino)

# xTreino = scaler.transform(xTreino)
# xTeste = scaler.transform(xTeste)

### Treinamento do modelo

Classificador Bayesiano com Distribuição Multinomial:

In [12]:
classificador = MultinomialNB(alpha=1.0)
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Acurácia = 0.4993


Classificador Bayesiano com Distribuição Multinomial:

In [13]:
classificador = GaussianNB()
y_pred = cross_val_predict(classificador, x, y, cv=5)

print("Acurácia = %6.4f" % accuracy_score(y,y_pred))

Acurácia = 0.4980


Classificador KNN

In [14]:
# print("\n  K  TREINO  TESTE")
# print(" --  ------  ------")

# for k in range(1,50):
#     classificador = KNeighborsClassifier(
#         n_neighbors=k,
#         weights = 'uniform',
#         p=1
#     )
#     classificador = classificador.fit(xTreino, yTreino)
    
#     yRespostaTreino = classificador.predict(xTreino)
#     yRespostaTeste = classificador.predict(xTeste)
    
#     acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
#     acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
#     print(
#         "%3d" % k,
#         "%6.1f" % (100*acuraciaTreino),
#         "%6.1f" % (100*acuraciaTeste)
#     )
for k in range(1,50,2):
    classificador = KNeighborsClassifier(n_neighbors=k)
    y_pred = cross_val_predict(classificador,x,y,cv=5)
    
    print("k = %2d ---> Acurácia = %6.4f" % (k, accuracy_score(y, y_pred) ) )


k =  1 ---> Acurácia = 0.5120
k =  3 ---> Acurácia = 0.5169
k =  5 ---> Acurácia = 0.5223
k =  7 ---> Acurácia = 0.5234
k =  9 ---> Acurácia = 0.5268
k = 11 ---> Acurácia = 0.5271
k = 13 ---> Acurácia = 0.5263
k = 15 ---> Acurácia = 0.5285
k = 17 ---> Acurácia = 0.5284
k = 19 ---> Acurácia = 0.5314
k = 21 ---> Acurácia = 0.5312
k = 23 ---> Acurácia = 0.5292
k = 25 ---> Acurácia = 0.5315
k = 27 ---> Acurácia = 0.5336
k = 29 ---> Acurácia = 0.5351
k = 31 ---> Acurácia = 0.5378
k = 33 ---> Acurácia = 0.5372
k = 35 ---> Acurácia = 0.5365
k = 37 ---> Acurácia = 0.5376
k = 39 ---> Acurácia = 0.5382
k = 41 ---> Acurácia = 0.5385
k = 43 ---> Acurácia = 0.5378
k = 45 ---> Acurácia = 0.5397
k = 47 ---> Acurácia = 0.5393
k = 49 ---> Acurácia = 0.5371


Classificador de Regressão Logística

In [71]:
# print("\n                        C  TREINO  TESTE")
# print(" -------------------------  ------  ------")

# for k in range(-6,7):
#     c = pow(10,k)
#     classificador = LogisticRegression(
#         penalty = 'l1',
#         C = c,
#         solver = 'liblinear',
#         max_iter = 10000
#     )

#     classificador = classificador.fit(xTreino, yTreino)
    
#     yRespostaTreino = classificador.predict(xTreino)
#     yRespostaTeste = classificador.predict(xTeste)
    
#     acuraciaTreino = sum(yRespostaTreino==yTreino)/len(yTreino)
#     acuraciaTeste = sum(yRespostaTeste==yTeste)/len(yTeste)
    
#     print(
#         "%14.6f" % c,
#         "%6.1f" % (100*acuraciaTreino),
#         "%6.1f" % (100*acuraciaTeste)
#     )

classificador = LogisticRegression(penalty='l2', C=10.0, solver='liblinear', max_iter=10000)
y_pred = cross_val_predict(classificador,x,y,cv=5)

print("Acurácia - %6.4f" % accuracy_score(y, y_pred))

Acurácia - 0.5803
